In [1]:
import torch

In [2]:
sem_classes = ['__background__', 'cable', 'tower_lattice', 'tower_tucohy', 'tower_wooden']
sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)}

In [3]:
from torchvision.models.segmentation import deeplabv3_resnet50

model = torch.hub.load('pytorch/vision:v0.11.0', 'deeplabv3_resnet50', pretrained=True)

Using cache found in /Users/thanhhung/.cache/torch/hub/pytorch_vision_v0.11.0
/Users/thanhhung/anaconda3/envs/vqi_v2/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/thanhhung/anaconda3/envs/vqi_v2/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
PATH = f'model/segmentation/deeplabv3_scripted.ptl'
if torch.cuda.is_available():
    model.load_state_dict(torch.load(PATH))
else:
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))

model = model.eval()

/Users/thanhhung/anaconda3/envs/vqi_v2/lib/python3.11/site-packages/torch/serialization.py:799: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


TypeError: Expected state_dict to be dict-like, got <class 'torch.jit._script.RecursiveScriptModule'>.

In [16]:
from model.segmentation.segmentation_output_wrapper import SegmentationModelOutputWrapper
model = SegmentationModelOutputWrapper(model)

In [10]:
from torch.utils.mobile_optimizer import optimize_for_mobile
scripted_module = torch.jit.script(model)
optimized_model = optimize_for_mobile(scripted_module)
# optimized_model.save("ImageSegmentation/deeplabv3_scripted.pt")
optimized_model._save_for_lite_interpreter("model/segmentation/DeepLabv3_ResNet50_scripted.ptl")